# SCENARI D'USO

In [1]:
import carla, time, threading, random
from agents.navigation.basic_agent import BasicAgent
from Adas import Forward_collision_warning_mqtt
from SyncSimulation import SyncSimulation

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()

# FUNZIONI DI SUPPORTO

## Gestione simulazione

In [4]:
# Simulazione sensore umidità dell'asflato
def get_asphalt_friction_coefficient():
    world = client.get_world()
    weather = world.get_weather()
    weather.precipitation_deposits
    weather.wetness
    return 1

In [42]:
def get_waypoint(location):
    map = world.get_map()
    return map.get_waypoint(location)

## ADAS

In [9]:
def attach_adas(sim, world, vehicle, min_ttc = 0.5):
    adas = Forward_collision_warning_mqtt(
        world = world,
        attached_vehicle = vehicle,
        min_ttc=min_ttc,
        get_asphalt_friction_coefficient = lambda : get_asphalt_friction_coefficient(),
        action_listener = lambda : sim.stop_simulation()
    )
    return adas

## Gestione world

In [10]:
def load_map(world, map_name):
    if not world.get_map().name == f"Carla/Maps/{map_name}":
        world = client.load_world(map_name)

In [11]:
def destroy_all_vehicles(world):
    all_actors = world.get_actors()
    
    vehicles = all_actors.filter('vehicle.*')
    
    print(f"Numero di veicoli trovati: {len(vehicles)}")
    
    for vehicle in vehicles:
        vehicle.destroy()
        print(f"Veicolo distrutto: {vehicle.id}")

In [12]:
def destroy_all_pedestrians(world):
    all_actors = world.get_actors()
    
    pedestrians = all_actors.filter('walker.pedestrian.*')
    
    print(f"Numero di pedoni trovati: {len(pedestrians)}")
    
    for pedestrian in pedestrians:
        pedestrian.destroy()
        print(f"Pedone distrutto: {pedestrian.id}")

In [13]:
def disable_traffic_lights():
    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.set_state(carla.TrafficLightState.Green)  # Imposta su verde
        traffic_light.set_green_time(99999)  # Mantiene il verde per un tempo molto lungo
        traffic_light.freeze(True)  # Disabilita la logica automatica

In [14]:
def enable_traffic_lights():

    traffic_lights = world.get_actors().filter('traffic.traffic_light')
    for traffic_light in traffic_lights:
        traffic_light.freeze(False)  # Riattiva il controllo automatico
        traffic_light.reset_group()  # Resetta lo stato e il ciclo del semaforo
        print(f"Semaforo {traffic_light.id} ripristinato al comportamento predefinito.")

## Creazione entità

In [15]:
def spawn_vehicle(world, location, index, y = 0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[index]
    spawn_point = carla.Transform()
    spawn_point.location = location
    spawn_point.rotation = carla.Rotation(yaw = y)
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    return vehicle

In [16]:
def actor_vehicle(world, vehicle, desitination, speed = 60):
    world.wait_for_tick()
    actor_agent = BasicAgent(vehicle)
    actor_agent.set_destination(desitination)
    actor_agent.set_target_speed(speed)
    actor_agent.ignore_vehicles()
    world.wait_for_tick()
    return actor_agent

In [17]:
def spawn_walker(start_location, yaw = 0):
    walker_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
    walker_bp.set_attribute('is_invincible', 'false')

    trans = carla.Transform()
    trans.location = start_location
    trans.rotation = carla.Rotation(yaw = yaw)

    return world.spawn_actor(walker_bp, trans)

## Gestione movimenti

In [18]:
def move_forward(vehicle, throttle):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)

In [19]:
def move_agent(vehicle, actor_agent):
    vehicle.apply_control(actor_agent.run_step())

In [20]:
def move_forward_agent(vehicle, actor_agent,x,y):
    forward_location = vehicle.get_location() + carla.Location(x=x, y=y, z=0)  # Adjust 'x', 'y' based on vehicle's orientation
    actor_agent.set_destination(forward_location)
    vehicle.apply_control(actor_agent.run_step())

In [21]:
def stop_vehicle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    control.hand_brake = True
    vehicle.apply_control(control)

In [22]:
def move_pedestrian(x=1,y =0,z = 0):
    walker_control = carla.WalkerControl()
    walker_control.speed = 1.5  # Velocità in m/s (positivo in avanti)
    walker_control.direction = carla.Vector3D(x, y, z)
    return walker_control

In [65]:
def stop_pedestrian(walker):
    walker_control = carla.WalkerControl()
    walker_control.speed = 0  # Velocità in m/s (positivo in avanti)
    walker.apply_control(walker_control)

In [23]:
def get_vehicle_velocity(vehicle):
    velocity_vector = vehicle.get_velocity()
    velocity = (velocity_vector.x**2 + velocity_vector.y**2 + velocity_vector.z**2)**0.5
    if vehicle.get_control().reverse:
        velocity = -velocity
    return velocity

## Camera

In [24]:
def print_spectator_position():
    location = world.get_spectator().get_transform().location
    rotation = world.get_spectator().get_transform().rotation
    print(location)
    print(rotation)

In [25]:
def set_spectator_location(location, rotation):
    spectator = world.get_spectator()
    spectator.set_transform(carla.Transform(location, rotation))

In [26]:
def run_camera(camera, vehicle, sim):
    return threading.Thread(target=camera, args=(vehicle,sim))

In [27]:
def view_from_above(vehicle, sim):    
    while sim.run:
        distance_above = 50.0  # Altezza della telecamera sopra il veicolo
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        
        
        spectator_location = carla.Location(
                x=vehicle_location.x,  # Stessa posizione X del veicolo
                y=vehicle_location.y,  # Stessa posizione Y del veicolo
                z=vehicle_location.z + distance_above  # Posizionata più in alto rispetto al veicolo
            )

        spectator_rotation = carla.Rotation(
            pitch=-90.0,  # Guardare direttamente verso il basso
            yaw=0,  # Allineato con la rotazione del veicolo
            roll=0.0
        )
        
        spectator.set_transform(carla.Transform(spectator_location, spectator_rotation))
    

In [28]:
def view_from_behind(vehicle, sim):    
    while sim.run:
        distance_behind = 14.0  # How far behind the car the spectator will be
        height = 5.0  # Height of the camera (how high above the car it will be)
        distance_to_the_side = 0
        
        spectator = world.get_spectator()
        
        vehicle_location = vehicle.get_location()
        vehicle_rotation = vehicle.get_transform().rotation
        
        spectator_location = carla.Location(
                x=vehicle_location.x - distance_behind * vehicle_rotation.get_forward_vector().x + distance_to_the_side * vehicle_rotation.get_right_vector().x,
                y=vehicle_location.y - distance_behind * vehicle_rotation.get_forward_vector().y + distance_to_the_side * vehicle_rotation.get_right_vector().y,
                z=vehicle_location.z + height  # Keeping the camera above the vehicle
            )

            # Set the spectator's transform to follow the vehicle
    
        spectator.set_transform(carla.Transform(spectator_location, vehicle_rotation))
    
    

# Scenari

## Scenario 0: Viaggio senza ostacoli


Verificare che in condiziondi normali il veicolo viaggi tranquillamente

### SETUP

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [ ]:
start = carla.Location(x=-41.477230, y=116.999939, z=0.000000)

# percorso da seguire
wps = [
    carla.Location(x=-41.679836, y=45.000507, z=0),
    carla.Location(x=30.000090, y=28.267391, z=0),
    carla.Location(x=-41.525066, y=100.000069, z=0)

]

### run

In [ ]:
destroy_all_vehicles(world)
simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, start, 0, -90)

actor_agent = actor_vehicle(world, vehicle, wps[0], 40)

wp_index = 0

try:
    adas = attach_adas(simulation, world, vehicle)
    simulation.run_sync_simulation()
    run_camera(view_from_behind, vehicle, simulation).start()
    
    while simulation.run:
        if actor_agent.done():
            wp_index = (wp_index +1) % len(wps)
            actor_agent.set_destination(wps[wp_index])

        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


## Scenario 1: Veicolo fermo o parcheggiato sulla carreggiata


Presenza di un veicolo parcheggiato direttamente davanti lungo la corsia di marcia.

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [ ]:
far_location = carla.Location(x=-41.5, y=45, z=1)
middle_location = carla.Location(x=-41.5, y=70, z=1)
close_location = carla.Location(x=-41.5, y=100, z=1)


In [ ]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=117, z=1), 2, -90)
actor_agent = actor_vehicle(world, vehicle, far_location, 120)

osbstacle_vehicle = spawn_vehicle(world, close_location, 1, -90)


try:
    adas = attach_adas(simulation, world, vehicle)
    simulation.run_sync_simulation()
    run_camera(view_from_above, vehicle, simulation).start()

    while simulation.run:
        move_agent(vehicle, actor_agent)

    stop_vehicle(vehicle)
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()



## Senario 2: Veicolo davanti in movimento con una velocità maggiore

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [43]:
# print_spectator_position()
print(get_waypoint(carla.Location(x=-41.453701, y=-42.637630, z=2.603971)))

Waypoint(Transform(Location(x=-41.647892, y=-42.639099, z=0.000000), Rotation(pitch=360.000000, yaw=270.432312, roll=0.000000)))


In [48]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

destination = carla.Location(x=-41.647892, y=-42.639099, z=0.000000)

vehicle_speed = 45
obstacle_speed = 50

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=117, z=1), 1, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=105, z=1), 1, -90)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, obstacle_speed)

try:
    adas = attach_adas(simulation, world, vehicle, 0.4)
    simulation.run_sync_simulation()  
    run_camera(view_from_above, vehicle, simulation).start()
    while simulation.run:
        if obstacle_actor_agent.done():
            stop_vehicle(obstacle_vehicle)
        vehicle.apply_control(actor_agent.run_step())
        obstacle_vehicle.apply_control(obstacle_actor_agent.run_step())

    
    stop_vehicle(vehicle)
    simulation.stop_simulation()
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(obstacle_vehicle)
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 2
Veicolo distrutto: 110
Veicolo distrutto: 111
simulation stopped
done


## Senario 3: Veicolo davanti in movimento con una velocità minore

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

In [56]:
destroy_all_vehicles(world)

simulation = SyncSimulation(world)

destination = carla.Location(x=30, y=28.3, z=1)

vehicle_speed = 60
obstacle_speed = 30

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=117, z=1), 1, -90)
actor_agent = actor_vehicle(world, vehicle, destination, vehicle_speed)

obstacle_vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=100, z=1), 1, -90)
obstacle_actor_agent = actor_vehicle(world, obstacle_vehicle, destination, obstacle_speed)

try:
    adas = attach_adas(simulation, world, vehicle, 0.4)
    simulation.run_sync_simulation()  
    run_camera(view_from_above, vehicle, simulation).start()
    while True:
        if obstacle_actor_agent.done():
            break
        if simulation.run:
            vehicle.apply_control(actor_agent.run_step())
        else:
            stop_vehicle(vehicle)
        obstacle_vehicle.apply_control(obstacle_actor_agent.run_step())

    
    stop_vehicle(obstacle_vehicle)
    simulation.stop_simulation()
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    adas.destroy()
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 2
Veicolo distrutto: 134
Veicolo distrutto: 135
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
simulation stopped
done


RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

## Scenario 4 : Pedone che attraversa le strisce pedonali


Presenza di un pedone che attraversa le strisce pedonali mentre il veicolo di riferimento è a pochi metri da esse

### SETUP

In [ ]:
load_map(world, "Town10HD_Opt")

In [ ]:
disable_traffic_lights()

### SIMULAZIONE SEMPLICE

In [61]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)

simulation = SyncSimulation(world)

destination = carla.Location(x=-41.5, y=10, z=1)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=80, z=1), 1, -90)
actor_agent = actor_vehicle(world, vehicle, destination, 120)

walker = spawn_walker(carla.Location(x=-41.5, y=37, z=1))


try:
    adas = attach_adas(simulation, world, vehicle)
    simulation.run_sync_simulation()
    set_spectator_location(carla.Location(x=-37.506443, y=33.672970, z=1.671275),
        carla.Rotation(pitch=-6.648561, yaw=118.152756, roll=-0.000091))

    while simulation.run:
        vehicle.apply_control(actor_agent.run_step())
            
    stop_vehicle(vehicle)
    
except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()



Numero di veicoli trovati: 1
Veicolo distrutto: 146
Numero di pedoni trovati: 1
Pedone distrutto: 147
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
done


### Pedone in movimento

In [66]:
destroy_all_vehicles(world)
destroy_all_pedestrians(world)


simulation = SyncSimulation(world)

destination = carla.Location(x=-41.5, y=72, z=1)

vehicle = spawn_vehicle(world, carla.Location(x=-41.5, y=80, z=1), 1, -90)
actor_agent = actor_vehicle(world, vehicle, destination, 35)

walker = spawn_walker(carla.Location(x=-35, y=36.6, z=1), 180)
controller_w = move_pedestrian(-1,0,0)

walker_should_move = False

try:
    adas = attach_adas(simulation, world, vehicle, 0.4)
    simulation.run_sync_simulation()
    run_camera(view_from_behind, vehicle, simulation).start()

    while simulation.run:
        if actor_agent.done():
            walker_should_move = True
            actor_agent.set_destination(carla.Location(x=-41.5, y=10, z=1))
        if walker_should_move:
            walker.apply_control(controller_w)
        vehicle.apply_control(actor_agent.run_step())
        
    stop_vehicle(vehicle)
    stop_pedestrian(walker)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()



Numero di veicoli trovati: 1
Veicolo distrutto: 157
Numero di pedoni trovati: 1
Pedone distrutto: 158
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
done


## Scenario 5: Salita

In [68]:
try:
    load_map(world, "Town03")
finally:
    print("load")

load


In [91]:
print(get_waypoint(carla.Location(x=153.095108, y=-58.077873, z=7)))

Waypoint(Transform(Location(x=152.688828, y=-58.084942, z=6.673837), Rotation(pitch=371.125397, yaw=270.996460, roll=0.000000)))


In [102]:
destroy_all_vehicles(world)
simulation = SyncSimulation(world)

destination = carla.Location(x=152.688828, y=-58.084942, z=6.673837)

vehicle = spawn_vehicle(world, carla.Location(x=178.759857, y=-5.810725, z=1), 1, 180)
actor_agent = actor_vehicle(world, vehicle, destination, 40)

close_location = carla.Location(x=152.025681, y=-19.958912, z=1)
middle_location = carla.Location(x=152.256927, y=-33.253975, z=3)
far_location = carla.Location(x=152.688828, y=-58.084942, z=7)

obstacle_vehicle = spawn_vehicle(world, far_location, 0, -90) 


stop_vehicle(obstacle_vehicle)

try:
    adas = attach_adas(simulation, world, vehicle)
    simulation.run_sync_simulation()
    run_camera(view_from_above, vehicle, simulation).start()
    
    while simulation.run:
        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 1
Veicolo distrutto: 277
simulation stopped
done


## Scenario 6: Salita inversa

In [97]:
try:
    load_map(world, "Town03")
finally:
    print("load")

load


In [147]:
disable_traffic_lights()

In [160]:
print_spectator_position()

Location(x=149.001038, y=-60.935722, z=7.710964)
Rotation(pitch=-38.835350, yaw=87.256798, roll=-0.000031)


In [161]:
print(get_waypoint(carla.Location(x=149.001038, y=-60.935722, z=7.710964)))

Waypoint(Transform(Location(x=149.237808, y=-60.931606, z=7.210840), Rotation(pitch=-10.385077, yaw=90.996475, roll=0.000000)))


In [178]:
destroy_all_vehicles(world)
close_location = carla.Location(x=149.237808, y=-56.931606, z=7)
middle_location = carla.Location(x=149.052383, y=-50.271507, z=6)
far_location = carla.Location(x=149.052383, y=-20.271507, z=2)

obstacle_vehicle = spawn_vehicle(world, far_location, 0, 90)
stop_vehicle(obstacle_vehicle)


Numero di veicoli trovati: 1
Veicolo distrutto: 369


In [164]:
destroy_all_vehicles(world)
vehicle = spawn_vehicle(world, carla.Location(x=149.401398, y=-100.336784, z=9.000000), 1, 90)

Numero di veicoli trovati: 2
Veicolo distrutto: 345
Veicolo distrutto: 346


In [179]:
destroy_all_vehicles(world)
simulation = SyncSimulation(world)

destination = carla.Location(x=111.904289, y=-6.809386, z=0.119281)


vehicle = spawn_vehicle(world, carla.Location(x=149.401398, y=-100.336784, z=9.000000), 1, 90)
actor_agent = actor_vehicle(world, vehicle, destination, 40)

close_location = carla.Location(x=149.237808, y=-56.931606, z=7)
middle_location = carla.Location(x=149.052383, y=-40.271507, z=6)
far_location = carla.Location(x=149.052383, y=-20.271507, z=2)

obstacle_vehicle = spawn_vehicle(world, far_location, 0, 90) 

stop_vehicle(obstacle_vehicle)

try:
    adas = attach_adas(simulation, world, vehicle)
    simulation.run_sync_simulation()
    run_camera(view_from_above, vehicle, simulation).start()
    
    while simulation.run:
        if actor_agent.done():
            simulation.stop_simulation()

        move_agent(vehicle, actor_agent)
        
    stop_vehicle(vehicle)

except KeyboardInterrupt:
    print("simulation stopped")
    stop_vehicle(vehicle)
    simulation.stop_simulation()
finally:
    print("done")
    adas.destroy()


Numero di veicoli trovati: 1
Veicolo distrutto: 370
Send `WARNING` to topic `carla/fcw_state`
Send `ACTION` to topic `carla/fcw_state`
done


## Scenario nuovo: svolta

In [181]:
try:
    load_map(world, "Town03")
finally:
    print("load")

load


In [182]:
disable_traffic_lights()

In [189]:
print_spectator_position()

Location(x=115.666473, y=-125.411621, z=9.217891)
Rotation(pitch=-47.727291, yaw=-3.183167, roll=0.000115)


In [190]:
print(get_waypoint(carla.Location(x=115.666473, y=-125.411621, z=9.217891)))

Waypoint(Transform(Location(x=115.813835, y=-132.290451, z=8.000000), Rotation(pitch=360.000000, yaw=1.227264, roll=0.000000)))


In [192]:
destroy_all_vehicles(world)

far_location = carla.Location(x=115.813835, y=-132.290451, z=9.000000)

obstacle_vehicle = spawn_vehicle(world, far_location, 0, 0)
stop_vehicle(obstacle_vehicle)


Numero di veicoli trovati: 1
Veicolo distrutto: 376


In [194]:
for waypoint in world.get_map().generate_waypoints(2.0):
    world.debug.draw_point(
        waypoint.transform.location,
        size=0.1,
        color=carla.Color(255, 0, 0),  # Colore rosso
        life_time=10.0  # Persistenza di 10 secondi
    )